In [20]:
import requests, pandas as pd, sqlite3 as sql3
from bs4 import BeautifulSoup

In [21]:
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'

extract_cols = ['Name', 'MC_USD_Billion']

In [22]:
r = requests.get(url)

In [23]:
html_parsed = BeautifulSoup(r.text, 'html.parser')
all_tables = html_parsed.find_all('tbody')
table = all_tables[0]

rows = table.find_all('tr')

df = pd.DataFrame(columns=extract_cols)
for row in rows:
    td_data = row.find_all('td')

    if len(td_data) != 0:
        x = td_data[1].find_all('a')[1]['title']
        y = td_data[2].contents[0][:-1]
        # print(y)
        df_data = pd.DataFrame([{'Name': td_data[1].find_all('a')[1]['title'],
                   'MC_USD_Billion': y
                }])
        df = pd.concat([df, df_data], ignore_index=True)

df.head(1)

,Name,MC_USD_Billion
0,JPMorgan Chase,432.92


In [24]:
d ='exchange_rate.csv'

In [25]:
rates = pd.read_csv(d)
rates_dict = rates.set_index('Currency').to_dict()['Rate']
df['MC_USD_Billion'] = df['MC_USD_Billion'].astype(float)
df['MC_GBP_Billion'] = df['MC_USD_Billion'].apply(lambda x : round(rates_dict['GBP'] * x,0))
df['MC_EUR_Billion'] = df['MC_USD_Billion'].apply(lambda x : round(rates_dict['EUR'] * x,2))
df['MC_INR_Billion'] = df['MC_USD_Billion'].apply(lambda x : round(rates_dict['INR'] * x,2))
df.head(2)

,Name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,JPMorgan Chase,432.92,346.0,402.62,35910.71
1,Bank of America,231.52,185.0,215.31,19204.58


In [71]:
df.head(1)
df['MC_EUR_Billion'][4]

np.float64(146.86)

In [74]:
db_name = 'world_economies.db'
table_name = 'countries_by_GDP'

sql3_connection = sql3.connect(db_name)

In [75]:
df.to_sql(sql3_connection, table_name, if_exists='replace')

ImportError: Using URI string without sqlalchemy installed.